In [1]:
import torch as tc
import numpy as np
import pandas as pd

from star.star import STAR
from curve_utils import CurveUtils
from curve_generator import CurveGenerator, get_all_curves, select_better
device = tc.device("cuda" if tc.cuda.is_available() else "cpu")

# Prepating Bodies

In [2]:
# %%script false --no-raise-error
for gender in ['female']:
    betas = tc.FloatTensor(np.load(f'data/{gender}_betas.npy')).to(device)
    trans = tc.FloatTensor(np.load(f'data/{gender}_trans.npy')).to(device)
    star = STAR(gender=gender, passed_path=f"star/{gender}.npz", num_betas=betas.shape[-1])
    
    poses = tc.FloatTensor(np.load(f'data/{gender}_poses.npy')).to(device)
    bodies = star(poses, betas, trans)
    tc.save(bodies, f'data/{gender}_bodies_a.pt')
    
    poses = np.zeros((betas.shape[0],72))
    poses[:,5] = 6 * np.pi/180
    poses[:,8] = -6 * np.pi/180
    poses = tc.cuda.FloatTensor(poses)
    bodies = star(poses, betas, trans)
    tc.save(bodies, f'data/{gender}_bodies_t.pt')

template = star.faces.to(device)

In [3]:
%%script false --no-raise-error
for gender in ['female', 'male']:
    bodies = tc.load(f'data/{gender}_bodies_a.pt')
    for idx, body in enumerate(bodies):
            CurveUtils.save_obj(f"tests/{gender}_{idx}_body.obj", body, star.faces+1)

# Loading Measures

In [4]:
their_semantics = [
    'Neck girth',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Upper arm girth R',
    'Thigh girth R',
    'Height (m)',
    'Inside leg height',
    'Across back shoulder width',
]
our_semantic = [
    'neck_girth', # 5.3.2
    'bust_chest_girth', # 5.3.4
    'waist_girth', # 5.3.10
    'hip_girth', # 5.3.13
    'upper_arm_girth', # 5.3.16
    'thigh_girth', # 5.3.20
    'height', # 5.1.1
    'crotch_height', # 5.1.15
    'shoulder_width', # 5.4.3
]

In [5]:
selected_subjects = dict()
selected_measures = dict()

for gender in ['female', 'male']:
    measures = pd.read_pickle(f'data/{gender}_measures.pkl')
    measures = measures[measures['Measuring station'] == "MOVE4D"]
    temp_measures = measures[their_semantics].iloc[::2]
    
    mean_measures = measures.mean(axis=0)
    # selected_subjects[gender] = abs(temp_measures - mean_measures).sum(axis=1).idxmin()
    selected_subjects[gender] = 'IEEEP2_01' if gender == 'female' else 'IEEEP2_02'
    selected_measures[gender] = temp_measures.loc[selected_subjects[gender]]
    selected_subjects[gender] = temp_measures.index.get_loc(selected_subjects[gender])
    selected_measures[gender].index = our_semantic
    selected_measures[gender]['height'] *= 1000

/tmp/ipykernel_953681/2761445603.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_measures = measures.mean(axis=0)
/tmp/ipykernel_953681/2761445603.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_measures = measures.mean(axis=0)


In [6]:

results = get_all_curves(selected_subjects, selected_measures, template, device)
for gender in ['female', 'male']:
    tc.save(results[gender], f'./data/{gender}_result.zip')

PROCESSING FEMALE
PROCESSING MALE


In [7]:
select_better(selected_subjects, selected_measures, device)


FEMALE
                  original    measured  error(mm)
neck_girth         32.7377   32.726025   0.116754
bust_chest_girth   89.1216    89.11258   0.090207
waist_girth        75.5405    75.53791   0.025895
hip_girth         105.7480  105.731155   0.168446
upper_arm_girth    28.5042    28.46891   0.352898
thigh_girth        62.9809    63.03246   0.515593
height            164.0000  165.284061  12.840614

MALE
                  original    measured  error(mm)
neck_girth         38.2454   38.256454   0.110545
bust_chest_girth  104.7790  104.813835   0.348351
waist_girth        88.6614    88.67702   0.156172
hip_girth         101.3170   101.28019   0.368105
upper_arm_girth    33.2674    33.27432   0.069187
thigh_girth        57.6963   57.589596   1.067042
height            173.0000  172.957659   0.423412


# Validation

In [8]:
curve_index = {
    'neck_girth':4, # 5.3.2
    'bust_chest_girth': 0, # 5.3.4
    'waist_girth': 1, # 5.3.10
    'hip_girth': 1, # 5.3.13
    'upper_arm_girth': 3, # 5.3.16
    'thigh_girth': 2, # 5.3.20
}

In [9]:
all_measures = dict()

for gender in ['female', 'male']:
    measures = pd.read_pickle(f'data/{gender}_measures.pkl')
    all_measures[gender] = dict()
    for station in ['MOVE4D', 'Expert measurer 1', 'Expert measurer 5']:
        temp_measures = measures[measures['Measuring station'] == station]
        temp_measures = temp_measures[their_semantics].iloc[::2]
        temp_measures = temp_measures[their_semantics]
        temp_measures.columns = our_semantic
        all_measures[gender][station] = temp_measures

In [10]:
tc.set_printoptions(precision=2, sci_mode=False, linewidth=100)

for gender in ['female', 'male']:
    bodies = tc.load(f'data/{gender}_bodies_a.pt')
    best_curves = tc.load(f'data/{gender}_best_curves.zip')

    all_bodies_measures = []
    for idx,body in enumerate(bodies):
        all_curves_measures = []
        all_positions = []
        for curve in best_curves:
            position = CurveGenerator.generate_positions(body, curve)[::2]
            all_positions.extend(position.cpu().numpy())
            measures = CurveGenerator.calculate_distances(position)
            all_curves_measures.append(measures)
        all_bodies_measures.append(all_curves_measures)
        CurveUtils.save_obj(f"tests/{gender}_{idx}_curves.obj", all_positions)
    all_bodies_measures = tc.FloatTensor(all_bodies_measures)
    our_measures = all_bodies_measures

    for station in ['MOVE4D', 'Expert measurer 1', 'Expert measurer 5']:
        print(f"\n\nmeasures difference from {station}:")
        inkreate_measures = tc.FloatTensor(all_measures[gender][station][list(curve_index.keys())].to_numpy())
        diff = abs(inkreate_measures[::2] - (our_measures*10))

        print(pd.DataFrame({
            "mean": diff.mean(axis=0),
            "std": diff.std(axis=0),
            "min": diff.min(axis=0).values,
            "max": diff.max(axis=0).values,
        }, index=list(curve_index.keys())))



measures difference from MOVE4D:
                       mean        std       min        max
neck_girth         8.962701   5.279659  1.419495  20.269348
bust_chest_girth  14.320072  11.964585  1.880798  39.682007
waist_girth       13.561870  11.516052  1.680298  39.929932
hip_girth         17.224409  17.291718  0.809692  62.872559
upper_arm_girth    5.779842   4.022608  0.938629  14.748718
thigh_girth       11.098097   5.097017  1.697266  21.107910


measures difference from Expert measurer 1:
                       mean        std       min        max
neck_girth        11.393088   9.181076  0.631592  35.647888
bust_chest_girth  18.130131  12.654757  1.673828  49.510986
waist_girth       29.253380  16.309570  0.500183  58.864807
hip_girth         17.977722   8.728536  3.179688  40.178101
upper_arm_girth    9.219637   7.090291  0.263885  24.891479
thigh_girth       13.781871   9.420030  0.603577  38.435913


measures difference from Expert measurer 5:
                       mean      